### Pregunta 4 - ¿Existen combinaciones de géneros o características (ej: "RPG + Open World") que correlacionen con tasas de abandono excepcionalmente altas/bajas?

#### Análisis preliminar
Para abordar este problema, será necesario utilizar matrices binarias al trabajar con variables categóricas como género, plataforma y tipo, ya que las dos primeras están representadas como cadenas de texto (strings). Esto implica un procesamiento adicional para evitar redundancias o solapamientos entre categorías similares.

Inicialmente, se identificaron 247 géneros únicos en el conjunto de datos de videojuegos. Tras un proceso de normalización manual —agrupando géneros similares y eliminando aquellos con menos de 5 apariciones— este número se redujo a 88 géneros relevantes. Este filtro asegura que solo se consideren géneros significativos para el análisis, reduciendo el ruido del conjunto.

Luego se define un criterio de tasa de abandono (*retired rate*), esta tasa representa la cantidad de jugadores que han abandonado un juego y se calcula como:

$$
\text{retired\_rate} = \frac{\text{count\_retired}}{\text{count\_total}}
$$

**Donde**:
- `count_retired`: Número de jugadores que abandonaron el juego.
- `count_total`: Número total de jugadores analizados.

**Criterio de abandono alto**:
Un juego se clasifica como de *alto abandono* si su tasa supera el **20%** (`retired_rate > 0.2`). Esta condición se traduce a una variable binaria:
- `1`: Alto abandono (cumple la condición).
- `0`: Abandono bajo o normal.

Esta métrica es clave para evaluar patrones entre los *clusters* (género, plataforma, etc.) y la tendencia al abandono. El objetivo final es identificar relaciones que expliquen por qué ciertos juegos tienen tasas de abandono más elevadas.

#### Reducción de dimensionalidad

El dataframe actual presenta un problema de **alta dimensionalidad** debido a la naturaleza de los datos. Al transformar las variables categóricas (*géneros*, *plataformas* y *tipos de juego*) en matrices binarias (one-hot encoding), el conjunto de datos resultante contiene un total de **217 columnas/features**. Para abordar este desafío, se implementaron **tres técnicas de reducción de dimensionalidad**:

1. **PCA (Análisis de Componentes Principales)**

2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**

3. **UMAP (Uniform Manifold Approximation and Projection)**

**Metodología de evaluación**:
- Para cada técnica, se probaron múltiples configuraciones:
  - Variación en número de componentes
  - Diferentes hiperparámetros

En este informe se presentan solo los parámetros elegidos donde se consideró mejor rendimiento, esto debido a limitaciones computacionales que era hacer cross-validation y probar más configuraciones cada vez que se quería alterar el código.



#### Selección de atributos de k-means

Para lograr seleccionar correctamente los atributos de k-means para cada caso, se ejecutó el clustering para k en el intervalo [1, 11], para luego decidir el valor óptimo de k utilizando el método del codo basándose en los siguientes gráficos.

<img src="img_p4/codo_pca.png" alt="Imagen 1" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/codo_umap.png" alt="Imagen 2" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/codo_t-sne.png" alt="Imagen 3" width="300" style="display:inline-block;"/>

#### Selección de atributos de DBSCAN

Para determinar un valor adecuado de eps, se utilizó una gráfica de distancias al k-ésimo vecino más cercano (con k = min_samples).

<img src="img_p4/umbral_pca.png" alt="Imagen 4" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/umbral_umap.png" alt="Imagen 5" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/umbral_t-sne.png" alt="Imagen 6" width="300" style="display:inline-block;"/>

Donde el valor marcado como "umbral" es el que se tiene que definir como radio.

#### Evaluación de clusters

Hasta este momento se han definido un total de 6 clusters, los cuales serán evaluados con las siguientes métricas

- Silhouette score: cohesión interna del cluster.

- Davies-Bouldin index: separación entre clusters (menor es mejor).

- Calinski-Harabasz index: relación varianza inter-cluster/intra-cluster.

A continuación se ven los resultados:

<img src="img_p4/evaluation.png" alt="Imagen 4" width="500" style="display:inline-block; margin-right:10px;"/>

Donde se ve claramente que el cluster "K-means con PCA" es superior en todas las métricas definidas, por lo tanto, es el que se continuará utilizando. Además, notar que "DBSCAN con PCA" no se logra armar ningún cluster, por lo tanto, no es evaluable.

#### Análisis  final

Para el Análisis  final es necesario aplicar las etiquetas generadas por el cluster seleccionado a el set de datos, luego de esto se puede analizar la distribución de juegos con alto abandono entre clusters, lo que lleva a las siguientes gráficas, donde el valor `high_retired` en 1 significa que es un juego con alto abandono.

<img src="img_p4/abandono1.png" alt="Imagen 4" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/abandono2.png" alt="Imagen 5" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/abandono3.png" alt="Imagen 5" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/abandono4.png" alt="Imagen 6" width="300" style="display:inline-block;"/>

Si bien este análisis no reveló un patrón concluyente que asocie clusters específicos con tasas de abandono excepcionalmente altas, logramos identificar un cluster dominante que muestra consistentemente una menor proporción de juegos clasificados como high_retired = 1. Este hallazgo preliminar es prometedor, ya que podría indicar una mayor retención de usuarios en los géneros o características que definen a este grupo. No obstante, para validar y profundizar en esta potencial relación, será fundamental aplicar herramientas estadísticas adicionales en futuros trabajos, lo que permitirá un estudio más exhaustivo de las combinaciones de géneros que favorecen la retención.